# Setting up the Fast Withdrawal contract with a proxy:
- This notebook allows deploying contracts for fast withdrawal interactions and running a basic scenario by creating a withdrawal on the Etherlink side and making fast payouts on the Tezos side.

In [1]:
from docs.scenarios.setup import *

web3, etherlink_account, tezos_account = setup()

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `236.925377 ꜩ`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `610.433834875 ꜩ`


## Deploy FastWithdrawal:

In [2]:
from scripts.tezos import deploy_fast_withdrawal

fast_withdrawal = deploy_fast_withdrawal.callback(
    exchanger_address=EXCHANGER_ADDRESS,
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    skip_confirm=True,
    silent=False,
)

Deploying Fast Withdrawal contract:
  - Deployer: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/ghostnet/`
  - Params:
      * Exchanger address: `KT1Bp9YUvUBJgXxf5UrYTM2CGRUPixURqx4m`
      * Smart Rollup address: `sr18wx6ezkeRjt1SZSeZ2UQzQN3Uc3YLMLqg`
Successfully deployed Fast Withdrawal, address: `KT1VfQrJktKDmwjE3e4rmQk7YMYuvLmsCCdP`


## Making Fast Withdrawal on Etherlink side:

In [3]:
from scripts.etherlink import xtz_fast_withdraw

xtz_fast_withdraw.callback(
    target=get_address(tezos_account),
    fast_withdrawal_contract=fast_withdrawal.address,
    amount=717 * 10**12,
    discounted_amount=700,
    withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Fast Withdrawal contract: `KT1VfQrJktKDmwjE3e4rmQk7YMYuvLmsCCdP`
      * Payload bytes: `0500bc0a`
      * Amount (mutez): `717`
      * Discounted amt (mutez): `700`
      * Fee (mutez): `17`
Successfully initiated XTZ Fast Withdrawal, tx hash: `0xa2374e92f646c589fdb3de22754c0ed9d197438b46da91c04b3cbfd854a8f126`


'0xa2374e92f646c589fdb3de22754c0ed9d197438b46da91c04b3cbfd854a8f126'

- Event logs: https://testnet.explorer.etherlink.com/tx/0xa2374e92f646c589fdb3de22754c0ed9d197438b46da91c04b3cbfd854a8f126?tab=logs
- Outbox message: https://ghostnet-smart.tzkt.io/global/block/11315946/outbox/11315946/messages
- Withdrawal ID: `1864`
- Timestamp: `1742548931`
- [ ] TODO: Retrieve `withdrawal_id` and `timestamp` from the Etherlink side event/outbox; currently, it is set manually.

## Purchasing withdrawal on Tezos side:

In [13]:
from scripts.helpers.contracts.fast_withdrawal import Withdrawal
from scripts.helpers.utility import pack
from scripts.helpers.ticket_content import TicketContent

discounted_amount = 700

withdrawal = Withdrawal(
    withdrawal_id=1864,
    full_amount=717,
    ticketer=EXCHANGER_ADDRESS,
    content=TicketContent(
        token_id=0,
        token_info=None,
    ),
    timestamp=1742548931,
    base_withdrawer=get_address(tezos_account),
    payload=pack(discounted_amount, 'nat'),
    l2_caller=bytes.fromhex(etherlink_account.address.split('0x')[1])
)

opg = fast_withdrawal.payout_withdrawal(
    withdrawal=withdrawal,
    service_provider=get_address(tezos_account),
    xtz_amount=discounted_amount,
).send()

opg.hash()

'ooL69vHKHxCwNbd1NFsz8Ge6S3zTmsrRBd3Ya8rqkMyTgyPrmFT'

- Transaction: https://ghostnet.tzkt.io/ooL69vHKHxCwNbd1NFsz8Ge6S3zTmsrRBd3Ya8rqkMyTgyPrmFT/27211184

## Trying to purchase twice:
- Forcing the transaction to be executed

In [17]:
opg = (
    fast_withdrawal.payout_withdrawal(
        withdrawal=withdrawal,
        service_provider=get_address(tezos_account),
        xtz_amount=discounted_amount,
    )
    .as_transaction()
    .fill()
    .sign()
    .inject()
)
opg['hash']

'oosddHjXw4gvREbLTNJ4HwMGPARZNxdzwN4w8qc7rjLPb7FxR3A'

## Making Additional Withdrawals:
### One with an incorrect payload that would be impossible to execute

In [18]:
from scripts.etherlink.xtz_fast_withdraw import (
    load_withdraw_precompile,
    make_fast_withdrawal,
)

precompile_contract = load_withdraw_precompile(
    XTZ_WITHDRAWAL_PRECOMPILE,
    web3
)
receipt = make_fast_withdrawal(
    etherlink_account=etherlink_account,
    web3=web3,
    precompile_contract=precompile_contract,
    target=get_address(tezos_account),
    fast_withdrawals_contract=fast_withdrawal.address,
    payload=bytes.fromhex('ff4f'),
    wei_amount=133 * 10**12,
)

click.echo(
    'Successfully initiated wrong XTZ Fast Withdrawal, tx hash: '
    + wrap(accent(receipt.transactionHash.hex()))
)

Successfully initiated wrong XTZ Fast Withdrawal, tx hash: `0x7d0d354c3c593c37181cc3cb1414d6d32c8613c927f2df6a5bb06d028291d720`


### ~~Three~~ Four more with different base withdrawers and amounts

In [19]:
from scripts.etherlink import xtz_fast_withdraw
from random import randint

xtz_fast_withdraw.callback(
    target=get_address(tezos_account),
    fast_withdrawal_contract=fast_withdrawal.address,
    amount=randint(100, 1000) * 10**12,
    discounted_amount=randint(100, 1000),
    withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Fast Withdrawal contract: `KT1VfQrJktKDmwjE3e4rmQk7YMYuvLmsCCdP`
      * Payload bytes: `0500a906`
      * Amount (mutez): `938`
      * Discounted amt (mutez): `425`
      * Fee (mutez): `513`
Successfully initiated XTZ Fast Withdrawal, tx hash: `0x2a8be2bc7d3b81a44d334e9c45d1b77beba0316c55b2d1184816b3bc45d7b21d`


'0x2a8be2bc7d3b81a44d334e9c45d1b77beba0316c55b2d1184816b3bc45d7b21d'

- 1866: [L2](https://testnet.explorer.etherlink.com/tx/0x2a8be2bc7d3b81a44d334e9c45d1b77beba0316c55b2d1184816b3bc45d7b21d), [outbox](https://ghostnet-smart.tzkt.io/global/block/11316307/outbox/11316307/messages), not executed


In [20]:
xtz_fast_withdraw.callback(
    target=tezos_account.key.generate(export=False).public_key_hash(),
    fast_withdrawal_contract=fast_withdrawal.address,
    amount=randint(100, 1000) * 10**12,
    discounted_amount=randint(100, 1000),
    withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1g55WPvhDFkWLDWB8M515FeX4DQZBPT1dd`
      * Fast Withdrawal contract: `KT1VfQrJktKDmwjE3e4rmQk7YMYuvLmsCCdP`
      * Payload bytes: `0500b40c`
      * Amount (mutez): `953`
      * Discounted amt (mutez): `820`
      * Fee (mutez): `133`
Successfully initiated XTZ Fast Withdrawal, tx hash: `0xaee04a7d43b9a3b626e1d5497b3059ca8ef477ec0176387f9eb883d20c67ae0b`


'0xaee04a7d43b9a3b626e1d5497b3059ca8ef477ec0176387f9eb883d20c67ae0b'

- 1868: [L2](https://testnet.explorer.etherlink.com/tx/0xaee04a7d43b9a3b626e1d5497b3059ca8ef477ec0176387f9eb883d20c67ae0b), [outbox](https://ghostnet-smart.tzkt.io/global/block/11316338/outbox/11316338/messages), not executed

In [21]:
xtz_fast_withdraw.callback(
    target=tezos_account.key.generate(export=False).public_key_hash(),
    fast_withdrawal_contract=fast_withdrawal.address,
    amount=randint(100, 1000) * 10**12,
    discounted_amount=randint(100, 1000),
    withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1hZk51goaVJxd2jLXLqvq7ySsdxUtpZK6r`
      * Fast Withdrawal contract: `KT1VfQrJktKDmwjE3e4rmQk7YMYuvLmsCCdP`
      * Payload bytes: `05008c0a`
      * Amount (mutez): `182`
      * Discounted amt (mutez): `652`
      * Fee (mutez): `-470`
Successfully initiated XTZ Fast Withdrawal, tx hash: `0xdb5dba47496f4b30581001402dc49e9a6a2a8053bebaffb92484c29c780d27c1`


'0xdb5dba47496f4b30581001402dc49e9a6a2a8053bebaffb92484c29c780d27c1'

- 1869: [L2](https://testnet.explorer.etherlink.com/tx/0xdb5dba47496f4b30581001402dc49e9a6a2a8053bebaffb92484c29c780d27c1), [outbox](https://ghostnet-smart.tzkt.io/global/block/11316338/outbox/11316338/messages), not executed

In [22]:
xtz_fast_withdraw.callback(
    target=tezos_account.key.generate(export=False).public_key_hash(),
    fast_withdrawal_contract=fast_withdrawal.address,
    amount=randint(100, 1000) * 10**12,
    discounted_amount=randint(100, 1000),
    withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1Rm2rh8CR2tJucZjaxWJWScCthE1W5KGd2`
      * Fast Withdrawal contract: `KT1VfQrJktKDmwjE3e4rmQk7YMYuvLmsCCdP`
      * Payload bytes: `0500b704`
      * Amount (mutez): `600`
      * Discounted amt (mutez): `311`
      * Fee (mutez): `289`
Successfully initiated XTZ Fast Withdrawal, tx hash: `0x586d829a7bfe5a0c26b684777ddd88060e217e06e34d05e755ce600e6d883427`


'0x586d829a7bfe5a0c26b684777ddd88060e217e06e34d05e755ce600e6d883427'

- 1816: [L2](httpshttps://testnet.explorer.etherlink.com/tx/0x586d829a7bfe5a0c26b684777ddd88060e217e06e34d05e755ce600e6d883427), [outbox](https://ghostnet-smart.tzkt.io/global/block/11316338/outbox/11316338/messages), not executed